In [ ]:
%load_ext autoreload
%autoreload 2

# Mongo

> MongoBuffer class

In [ ]:
#| default_exp storage.buffer.mongo

In [ ]:
#| export
from __future__ import annotations
import logging
from dataclasses import dataclass
from typing import Optional, Tuple
from zoneinfo import ZoneInfo
import numpy as np
import pandas as pd
from bson.codec_options import CodecOptions

In [ ]:
#| export
from tspace.config.db import DBConfig, db_config_servers_by_name
from tspace.config.drivers import Driver
from tspace.config.vehicles import Truck
from tspace.storage.pool.mongo import MongoPool
from tspace.data.location import locations_by_abbr
from tspace.data.core import (
    ObservationMeta,
    ObservationMetaCloud,
    ObservationMetaECU,
    PoolQuery,
    veos_lifetime_end_date,
    veos_lifetime_start_date,
)
from tspace.data.external.pandas_utils import (
    decode_episode_batch_to_padded_arrays,
    decode_mongo_episodes,
    decode_mongo_records,
)

In [ ]:
#| export
from tspace.storage.buffer.buffer import Buffer  # type: ignore

In [ ]:
#| export
@dataclass(kw_only=True)
class MongoBuffer(Buffer[pd.DataFrame]):
    """
    A Buffer connected with a MongoDB database pool

    Args:

        batch_size: number of documents for batch sampling
        driver: driver of the vehicle
        truck: subject vehicle
        meta: metadata of the observation
        db_config: config for mongodb (str) or DBConfig (dict)
        the key leads to a config with db_name and collection name with a switch for record or episode:
            - string for db server name
            - or string of the format "usr:password@host:port"
                for mongo_cluster:
                    Host="10.10.0.4",  # url for the database server
                    port="23000",  # port for the database server
                    user_name="admin",  # username for the database server
                    password="ty02ydhVqDj3QFjT",  # password for the database server
                    ==> mongo_key = "admin:ty02ydhVqDj3QFjT@10.10.0.4:23000"
        torque_table_row_names: list of torque table row names, e.g. ['r0, r1, ..., r9']
        query: query for the pool
        pool: pool of the database, a `MongoPool` instance
        logger: logger for the buffer
        dict_logger: dict logger for the buffer
    """

    batch_size: int  # 0
    driver: Driver  # field(default_factory=Driver)
    truck: Truck  # field(default_factory=Truck)
    meta: ObservationMeta  # field(default_factory=ObservationMeta)
    db_config: DBConfig  # db_config_default
    torque_table_row_names: list[str]  # field(default_factory=list)
    query: Optional[PoolQuery] = None  # field(default_factory=PoolQuery)
    pool: Optional[MongoPool] = None  # field(default_factory=MongoPool)
    logger: Optional[logging] = None
    dict_logger: Optional[dict] = None

    def __post_init__(self):
        """set logger and load pool"""
        self.logger = self.logger.getChild("main").getChild("mongo buffer")
        self.logger.propagate = True
        if not self.torque_table_row_names:
            self.torque_table_row_names = self.meta.get_torque_table_row_names()
        super().__post_init__()
        self.load()

    def load(self):
        """load pool from database"""
        if self.db_config.type == "RECORD":
            self.query = PoolQuery(
                vehicle=self.truck.vid,
                driver=self.driver.pid,
                episodestart_start=veos_lifetime_start_date,
                episodestart_end=veos_lifetime_end_date,
                timestamp_start=veos_lifetime_start_date,
                timestamp_end=veos_lifetime_end_date,
            )
        else:  # type is 'EPISODE'
            self.query = PoolQuery(
                vehicle=self.truck.vid,
                driver=self.driver.pid,
                episodestart_start=veos_lifetime_start_date,
                episodestart_end=veos_lifetime_end_date,
                seq_len_from=0,
                seq_len_to=int(1e9),
            )

        # MongoPool stores both record and episode data, just need to specify the query and sample
        self.pool = MongoPool(
            db_config=self.db_config,
            query=self.query,
            meta=self.meta,
            codec_option=CodecOptions(tz_aware=True),
            logger=self.logger,
            dict_logger=self.dict_logger,
        )

        if self.pool.cnt != 0:
            # check plot with input vehicle and driver
            batch_1 = self.pool.sample(size=1, query=self.query)
            if type(self.meta) is ObservationMetaCloud:
                meta_in_db = ObservationMetaCloud(
                    state_specs=batch_1["meta"].iloc[0]["state_specs"],
                    action_specs=batch_1["meta"].iloc[0]["action_specs"],
                    reward_specs=batch_1["meta"].iloc[0]["reward_specs"],
                    site=locations_by_abbr[batch_1["meta"].iloc[0]["site"]["abbr"]],
                )
            elif type(self.meta) is ObservationMetaECU:
                meta_in_db = ObservationMetaECU(
                    state_specs=batch_1["meta"].iloc[0]["state_specs"],
                    action_specs=batch_1["meta"].iloc[0]["action_specs"],
                    reward_specs=batch_1["meta"].iloc[0]["reward_specs"],
                    site=locations_by_abbr[batch_1["meta"].iloc[0]["site"]["abbr"]],
                )
            else:
                raise NotImplementedError(
                    f"meta type {type(self.meta)} not implemented"
                )
            assert self.meta.have_same_meta(
                meta_in_db
            ), f"meta in db is {batch_1[0].meta}, but meta in config is {self.meta}"
        else:
            self.logger.warning(
                f"Buffer is empty, create new database, buffer_count: {self.pool.cnt}, "
                f"db_config: {self.db_config}, query: {self.query}, "
                f"meta: {self.meta}",
                extra=self.dict_logger,
            )
        (
            num_states,
            num_actions,
        ) = self.meta.get_number_of_states_actions()  # the realtime number
        self.logger.info(
            f"Connected to MongoDB {self.db_config.database_name}, "
            f"collection {self.db_config.collection_name}, "
            f"record number {self.pool.cnt}, "
            f"num_states: {num_states}, num_actions: {num_actions}",
            extra=self.dict_logger,
        )

    def decode_batch_records(
        self, df: pd.DataFrame
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Sample a batch of records from the pool.

        Args:

            df: dataframe of sampled from the record collection of the pool

        Return:

            A quadruple of numpy arrays (states, actions, rewards, next_states)
        """

        df_states, df_actions, ser_rewards, df_nstates = decode_mongo_records(
            df, self.torque_table_row_names
        )

        idx = pd.IndexSlice
        npa_states = np.stack(
            [
                df_state.loc[
                    :,  # type: ignore
                    idx[
                        :, :, :, :, "state", ["velocity", "thrust", "brake"]
                    ],  # same order as inference !!!
                ].values.T.flatten()  # transpose values array to match inference!!!
                for df_state in df_states
            ]
        ).astype(np.float32)
        npa_actions = np.stack(
            [
                df_action.loc[
                    :, idx[:, :, :, :, "action", self.torque_table_row_names]  # type: ignore
                ].values.T.flatten()
                for df_action in df_actions
            ]
        ).astype(np.float32)
        npa_rewards = np.stack(
            [ser_reward.values.T.flatten() for ser_reward in ser_rewards]
        )
        npa_nstates = np.stack(
            [
                df_nstate.loc[
                    :, idx[:, :, :, :, "nstate", ["velocity", "thrust", "brake"]]  # type: ignore
                ].values.T.flatten()
                for df_nstate in df_nstates
            ]
        ).astype(np.float32)
        return npa_states, npa_actions, npa_rewards, npa_nstates

    def sample(self) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Sampling a batch of records or episodes from the pool.

        Decoding the batch observations from mongodb nested dicts to pandas dataframe
        flatten is used to adapt to model interface

        Return:

            A quadruple of numpy arrays (states, actions, rewards, next_states)
        """
        df = self.pool.sample(size=self.batch_size, query=self.query)

        if self.db_config.type == "RECORD":
            states, actions, rewards, nstates = self.decode_batch_records(df)
        else:  # if pool collection type is EPISODE
            df_episodes = decode_mongo_episodes(df)

            df_episodes_stripped = df_episodes[["state", "action", "reward", "nstate"]]
            (
                states,
                actions,
                rewards,
                nstates,
            ) = decode_episode_batch_to_padded_arrays(
                df_episodes_stripped, self.torque_table_row_names
            )

        return states, actions, rewards, nstates

    def find(self, idx):
        """
        find a record by index
        """
        return self.pool.find(idx)

    def close(self):
        """close the pool and the database"""
        self.pool.close()

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
show_doc(MongoBuffer.__post_init__)

In [ ]:
show_doc(MongoBuffer.load)

In [ ]:
show_doc(MongoBuffer.find)

In [ ]:
show_doc(MongoBuffer.close)

In [ ]:
show_doc(MongoBuffer.sample)

In [ ]:
show_doc(MongoBuffer.decode_batch_records)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()